In [83]:
#Regular 3 phase SFDI 
#OpenSFDI data
#calibrating system with reference phantoms given (blank images)
#getting absorption and scattering of three phase data after calibrating with blank images
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import matplotlib
%matplotlib qt

w = 0.00067 #mm (670nm)
sf = [0, 0.2] #spatial frequencies - mm^{-1}
p = [0, (2/3)*np.pi, (4/3)*np.pi]#for 3 phases[0, 120, 240]degrees

images = [] #raw data


def image_tif(sf, p): #read in raw data
    im1 = cv2.imread('im_1_' + str(sf+1) + '_' + str(p+1) + '.tif', 0)
    return im1.astype(np.double)

for i in range(len(sf)):
    for j in range(len(p)):        
        image_tif(i, j)
        images.append(image_tif(i, j))#put all raw data into images[]
        #print(j)  #just to confirm its been executed 
        



#Demodulation
def AC(var):
    return (2**(1/2)/3)*(((var[3]-var[4])**2 + (var[4]-var[5])**2 + (var[5]-var[3])**2)**(1/2))

def DC(var):
    return np.asarray((1/3)*(var[0]+var[1]+var[2]))

M_AC = AC(images) #AC modulation amplitude

M_DC = DC(images) #DC modulation amplitude



#we need to figure out diffuse reflectance of phantom given the optical properties
images2 = []
ran = [1, 2, 3, 4, 5, 6]

for i in range(len(ran)):
    image_tif2 = cv2.imread('im0{}.tif'.format(i+1))
    images2.append(image_tif2.astype(np.double))

#Plots of M_AC and M_DC
M_AC_ref = AC(images2)

M_DC_ref = DC(images2)

M_AC_ref2 = []
M_DC_ref2 = []
for i in range(M_AC_ref.shape[0]):
    for j in range(M_AC_ref.shape[1]):
        M_AC_ref2.append(M_AC_ref[i][j][0])
        M_DC_ref2.append(M_DC_ref[i][j][0])


M_AC_ref3 = np.reshape(M_AC_ref2, (M_AC.shape[0], M_AC.shape[1]))
M_DC_ref3 = np.reshape(M_DC_ref2, (M_DC.shape[0], M_DC.shape[1]))


#need R_d,ref,pred
mu_a = 0.0059 #/mm
mu_sp = 0.9748 #/mm
n = 1.4 #refractive index of tissue 


f2 = 0.2 #mm^-1
R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
A = (1-R_eff)/(2*(1+R_eff))
mu_tr = mu_a + mu_sp
ap = mu_sp/mu_tr
mu_effp = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

R_AC_ref = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))

        
          


f1 = 0 #mm^-1
mu_effp1 = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)
R_DC_ref = (3*A*ap)/(((mu_effp1/mu_tr)+1)*((mu_effp1/mu_tr)+3*A))


print(M_AC_ref.shape)
                
R_d_AC = (M_AC/M_AC_ref3)*R_AC_ref
R_d_DC = (M_DC/M_DC_ref3)*R_DC_ref

plt.subplot(221), plt.imshow(R_d_AC), plt.colorbar(), plt.title('AC'), plt.xticks([]),  plt.yticks([])
plt.subplot(222), plt.imshow(R_d_DC), plt.colorbar(), plt.title('DC'), plt.xticks([]),  plt.yticks([])
     
xi = []
for i in range(R_d_AC.shape[0]): #does row first and loops through all columns, moves onto next row
    for j in range(R_d_AC.shape[1]):
        freq = [R_d_DC[i][j], R_d_AC[i][j]]
        xi.append(freq)
    

#Getting array of reflectance values and corresponding optical properties
mu_a = np.arange(0, 0.3, 0.001)
mu_sp = np.arange(0.3, 3, 0.01)
n = 1.4 #refractive index of tissue 

Reflectance_AC = []

for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f2 = 0.2 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

        R_d1 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)
        
          
Reflectance_DC = []
op_mua = []
op_sp = []

for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f1 = 0 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)

        R_d2 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)
                
        op2 = mu_a[i]
        op_mua.append(op2)
        
        op3 = mu_sp[j]
        op_sp.append(op3)


points = []

for k in range(len(mu_a)*len(mu_sp)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_array = np.array(points)

op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

values1 = op_mua_array
values2 = op_sp_array


coeff_abs3 = griddata(points_array, values1, xi, method='cubic') #mua
coeff_sct3 = griddata(points_array, values2, xi, method='cubic') #musp

abs_plot3 = np.reshape(coeff_abs3, (R_d_AC.shape[0], R_d_AC.shape[1]))
sct_plot3 = np.reshape(coeff_sct3, (R_d_AC.shape[0], R_d_AC.shape[1]))


plt.subplot(223), plt.imshow(abs_plot3), plt.title('Absorption $\mu_a$'), plt.xticks([]),  plt.yticks([])
plt.colorbar()

plt.subplot(224), plt.imshow(sct_plot3), plt.title('Reduced scattering $\mu_s$`'), plt.xticks([]),  plt.yticks([])
plt.colorbar()


(512, 672, 3)


D:\Users\janeh\anaconda\lib\site-packages\ipykernel_launcher.py:98: RuntimeWarning: divide by zero encountered in true_divide
D:\Users\janeh\anaconda\lib\site-packages\ipykernel_launcher.py:98: RuntimeWarning: invalid value encountered in true_divide


In [2]:
#Filter & shift
#SSOP to get optical properties
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
from unwrap import unwrap
from mpl_toolkits import mplot3d
import matplotlib
import scipy
import scipy.ndimage
from scipy.signal import hilbert
from PIL import Image
from scipy.interpolate import griddata

%matplotlib qt

im1 = (cv2.imread('im_1_2_1.tif',0)).astype(np.double)

height = int(im1.shape[0]) 
width = int(im1.shape[1])
row = int(im1.shape[0]/2)
col = int(im1.shape[1]/2) 

f1 = np.fft.fft2(im1) #2D DFT
#for high pass filter
FT1 = np.fft.fftshift(f1)
FT1p = 20*np.log(np.abs(FT1))
#for low pass filter
FT2 = np.fft.fftshift(f1)
FT2p = 20*np.log(np.abs(FT2))


FT_pd1 = pd.DataFrame(FT1p) #whole image
cbp1 = (np.where(FT_pd1 == np.max(FT1p))[1][0])
print("CBP: ", cbp1)
FT_maxrow1 = FT1p[np.where(FT_pd1 == np.max(FT1p))[1][0]]
num1 = np.where(FT_pd1 == np.max(FT1p))[1][0]
print("num1: ", num1)
lbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[0:num1-5]))[0][0] #left bright point
print("LBP: ", lbp1)
rbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[num1+4:width]))[0][0] #right bright point
print("RBP: ", rbp1)
diff1 = int(rbp1 - cbp1)
print("Difference: ", diff1)

#FOR PLOTTING
#HPF
mask0 = FT1p 
mask0[0:height, rbp1+int(diff1/2):rbp1+int(diff1/2)+int(2*diff1)] = 0
mask0[0:height, lbp1-int(diff1/2):cbp1+int(diff1/2)] = 0
roll0 = np.roll(mask0, -diff1)
#plt.subplot(221), plt.imshow(roll0)

#LPF
mask0l = FT2p
mask0l[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0
mask0l[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0
#plt.subplot(223), plt.imshow(mask0l)



#FOR IMAGE
#HIGH PASS FILTER
mask1 = FT1 
mask1[0:height, rbp1+int(diff1/2):rbp1+int(diff1/2)+int(2*diff1)] = 0
mask1[0:height, lbp1-int(diff1/2):cbp1+int(diff1/2)] = 0
roll1 = np.roll(mask1, -diff1)
IFT = np.abs((np.fft.ifft2(np.fft.ifftshift(roll1)))) #inverse FFT
#plt.subplot(222), plt.imshow(IFT), plt.colorbar()

#LOW PASS FILTER
maskl2 = FT2
maskl2[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0
maskl2[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0
IFT2 = np.abs((np.fft.ifft2(np.fft.ifftshift(maskl2))))
#plt.subplot(224), plt.imshow(IFT2), plt.colorbar()




#we need to figure out diffuse reflectance of phantom given the optical properties
ref_im = (cv2.imread('im04.tif',0)).astype(np.double)
f_ref = np.fft.fft2(ref_im) #2D DFT
#for high pass filter
FTh = np.fft.fftshift(f_ref)
FThp = 20*np.log(np.abs(FTh))
#for low pass filter
FTl = np.fft.fftshift(f_ref)
FTlp = 20*np.log(np.abs(FTl))


#FOR PLOTTING
#HPF
maska = FThp 
maska[0:height, rbp1+int(diff1/2):rbp1+int(diff1/2)+int(2*diff1)] = 0
maska[0:height, lbp1-int(diff1/2):cbp1+int(diff1/2)] = 0
rolla = np.roll(maska, -diff1)
#plt.subplot(221), plt.imshow(rolla)

#LPF
maskb = FTlp
maskb[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0
maskb[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0
#plt.subplot(223), plt.imshow(maskb)



#FOR IMAGE
#HIGH PASS FILTER
maskc = FTh 
maskc[0:height, rbp1+int(diff1/2):rbp1+int(diff1/2)+int(2*diff1)] = 0
maskc[0:height, lbp1-int(diff1/2):cbp1+int(diff1/2)] = 0
rollc = np.roll(maskc, -diff1)
IFT_h = np.abs((np.fft.ifft2(np.fft.ifftshift(rollc)))) #inverse FFT
#plt.subplot(222), plt.imshow(IFT_h), plt.colorbar() #this is M_AC ref

#LOW PASS FILTER
maskd = FTl
maskd[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0
maskd[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0
IFT2_l = np.abs((np.fft.ifft2(np.fft.ifftshift(maskd))))
#plt.subplot(224), plt.imshow(IFT2_l), plt.colorbar() #this is M_DC ref


#to get R_d ref
mu_a = 0.0059 #/mm
mu_sp = 0.9748 #/mm
n = 1.4 #refractive index of tissue 

f2 = 0.2 #mm^-1
R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
A = (1-R_eff)/(2*(1+R_eff))
mu_tr = mu_a + mu_sp
ap = mu_sp/mu_tr
mu_effp = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)
R_AC_ref_s = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))

        
f1 = 0 #mm^-1
mu_effp1 = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)
R_DC_ref_s = (3*A*ap)/(((mu_effp1/mu_tr)+1)*((mu_effp1/mu_tr)+3*A))

                
R_d_AC_S = (IFT/IFT_h)*R_AC_ref_s
R_d_DC_S = (IFT2/IFT2_l)*R_DC_ref_s

plt.subplot(221), plt.imshow(R_d_AC_S), plt.colorbar(), plt.title('AC'), plt.clim(0,0.4), plt.xticks([]), plt.yticks([])
plt.subplot(222), plt.imshow(R_d_DC_S), plt.colorbar(), plt.title('DC'), plt.xticks([]), plt.yticks([])




#GETTING OPTICAL PROEPRTIES
xi = []
for i in range(R_d_DC_S.shape[0]): #does row first and loops through all columns, moves onto next row
    for j in range(R_d_DC_S.shape[1]):
        freq = [R_d_DC_S[i][j], R_d_AC_S[i][j]]
        xi.append(freq)
    

#Getting array of reflectance values and corresponding optical properties
mu_a = np.arange(0, 0.3, 0.001)
mu_sp = np.arange(0.3, 3, 0.01)
n = 1.4 #refractive index of tissue 

Reflectance_AC = []
for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f2 = 0.2 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

        R_d1 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)
        
          
Reflectance_DC = []
op_mua = []
op_sp = []
for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f1 = 0 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)

        R_d2 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)
                
        op2 = mu_a[i]
        op_mua.append(op2)
        
        op3 = mu_sp[j]
        op_sp.append(op3)


points = []

for k in range(len(mu_a)*len(mu_sp)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_array = np.array(points)

op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

values1 = op_mua_array
values2 = op_sp_array


coeff_abs3 = griddata(points_array, values1, xi, method='cubic') #mua
coeff_sct3 = griddata(points_array, values2, xi, method='cubic') #musp

abs_plot3 = np.reshape(coeff_abs3, (R_d_DC_S.shape[0], R_d_DC_S.shape[1]))
sct_plot3 = np.reshape(coeff_sct3, (R_d_DC_S.shape[0], R_d_DC_S.shape[1]))



plt.subplot(223), plt.imshow(abs_plot3), plt.title('Absorption $\mu_a$'), plt.xticks([]),  plt.yticks([]), plt.clim(0, 0.15)
plt.colorbar()

plt.subplot(224), plt.imshow(sct_plot3), plt.title('Reduced Scattering $\mu_s$`'), plt.xticks([]),  plt.yticks([])
plt.colorbar()


CBP:  336
num1:  336
LBP:  308
RBP:  364
Difference:  28


In [5]:
#Georges way of doing hilbert
#SSOP to get optical properties
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
from unwrap import unwrap
from mpl_toolkits import mplot3d
import matplotlib
import scipy
import scipy.ndimage
from scipy.signal import hilbert
from PIL import Image
from scipy.interpolate import griddata

%matplotlib qt

im1 = (cv2.imread('im_1_2_1.tif',0)).astype(np.double)

height = int(im1.shape[0]) 
width = int(im1.shape[1])
row = int(im1.shape[0]/2)
col = int(im1.shape[1]/2) 

f1 = np.fft.fft2(im1) #2D DFT
#for high pass filter
FT1 = np.fft.fftshift(f1)
FT1p = 20*np.log(np.abs(FT1))
#for low pass filter
FT2 = np.fft.fftshift(f1)
FT2p = 20*np.log(np.abs(FT2))


FT_pd1 = pd.DataFrame(FT1p) #whole image
cbp1 = (np.where(FT_pd1 == np.max(FT1p))[1][0])
print("CBP: ", cbp1)
FT_maxrow1 = FT1p[np.where(FT_pd1 == np.max(FT1p))[1][0]]
num1 = np.where(FT_pd1 == np.max(FT1p))[1][0]
print("num1: ", num1)
lbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[0:num1-5]))[0][0] #left bright point
print("LBP: ", lbp1)
rbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[num1+4:width]))[0][0] #right bright point
print("RBP: ", rbp1)
diff1 = int(rbp1 - cbp1)
print("Difference: ", diff1)



#FOR IMAGE
#HIGH PASS FILTER
mask1 = FT1 
mask1[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0
IFT_AC = np.fft.ifft2(np.fft.ifftshift(mask1)) #inverse FFT

Real = np.real(IFT_AC)
Imag = np.imag(IFT_AC)

Hilb_r = scipy.signal.hilbert(Real, axis=1)
Hilb_i = scipy.signal.hilbert(Imag, axis=1)

Signal = (Hilb_r + 1j*Hilb_i)/2

AC_sig = np.abs(Signal)#/(np.max(np.abs(Signal))/0.3)
#plt.subplot(221), plt.imshow(AC_sig), plt.title('AC'), plt.colorbar()


#LOW PASS FILTER
maskl2 = FT2
maskl2[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0
maskl2[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0

IFT_DC = (np.fft.ifft2(np.fft.ifftshift(maskl2)))
DC_sig = np.abs(IFT_DC)#/np.max(IFT2)
#plt.subplot(222), plt.imshow(DC_plot), plt.title('DC'), plt.colorbar()



#we need to figure out diffuse reflectance of phantom given the optical properties
ref_im = (cv2.imread('im04.tif',0)).astype(np.double)
f_ref = np.fft.fft2(ref_im) #2D DFT
#for high pass filter
FTh = np.fft.fftshift(f_ref)
FThp = 20*np.log(np.abs(FTh))
#for low pass filter
FTl = np.fft.fftshift(f_ref)
FTlp = 20*np.log(np.abs(FTl))


#FOR IMAGE
#HIGH PASS FILTER
mask_AC = FTh 
mask_AC[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0
IFT_h = np.fft.ifft2(np.fft.ifftshift(mask_AC)) #inverse FFT

Real_ref = np.real(IFT_h)
Imag_ref = np.imag(IFT_h)

Hilb_r_ref = scipy.signal.hilbert(Real_ref, axis=1)
Hilb_i_ref = scipy.signal.hilbert(Imag_ref, axis=1)

Signal_ref = (Hilb_r_ref + 1j*Hilb_i_ref)/2

AC_sig_ref = np.abs(Signal_ref)

#LOW PASS FILTER
mask_DC = FTl
mask_DC[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0
mask_DC[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0
IFT2_DC = np.fft.ifft2(np.fft.ifftshift(mask_DC))
DC_sig_ref = np.abs(IFT2_DC)


#to get R_d ref
mu_a = 0.0059 #/mm
mu_sp = 0.9748 #/mm
n = 1.4 #refractive index of tissue 

f2 = 0.2 #mm^-1
R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
A = (1-R_eff)/(2*(1+R_eff))
mu_tr = mu_a + mu_sp
ap = mu_sp/mu_tr
mu_effp = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)
R_AC_ref_sh = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))

        
f1 = 0 #mm^-1
mu_effp1 = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)
R_DC_ref_sh = (3*A*ap)/(((mu_effp1/mu_tr)+1)*((mu_effp1/mu_tr)+3*A))

                
R_d_AC_Sh = (AC_sig/AC_sig_ref)*R_AC_ref_sh
R_d_DC_Sh = (DC_sig/DC_sig_ref)*R_DC_ref_sh

plt.subplot(221), plt.imshow(R_d_AC_Sh), plt.title('AC'), plt.colorbar(), plt.clim(0,0.4), plt.xticks([]), plt.yticks([])
plt.subplot(222), plt.imshow(R_d_DC_Sh), plt.title('DC'), plt.colorbar(), plt.xticks([]), plt.yticks([])


xi = []
for i in range(R_d_AC_Sh.shape[0]): #does row first and loops through all columns, moves onto next row
    for j in range(R_d_AC_Sh.shape[1]):
        freq = [R_d_DC_Sh[i][j], R_d_AC_Sh[i][j]]
        xi.append(freq)
    

#Getting array of reflectance values and corresponding optical properties
mu_a = np.arange(0, 0.3, 0.001)
mu_sp = np.arange(0.3, 3, 0.01)
n = 1.4 #refractive index of tissue 

Reflectance_AC = []

for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f2 = 0.2 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

        R_d1 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)
        
          
Reflectance_DC = []
op_mua = []
op_sp = []

for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f1 = 0 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)

        R_d2 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)
                
        op2 = mu_a[i]
        op_mua.append(op2)
        
        op3 = mu_sp[j]
        op_sp.append(op3)


points = []

for k in range(len(mu_a)*len(mu_sp)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_arrayh = np.array(points)

op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

values1 = op_mua_array
values2 = op_sp_array


coeff_abs3h = griddata(points_arrayh, values1, xi, method='cubic') #mua
coeff_sct3h = griddata(points_arrayh, values2, xi, method='cubic') #musp

abs_plot3h = np.reshape(coeff_abs3h, (R_d_AC_Sh.shape[0], R_d_AC_Sh.shape[1]))
sct_plot3h = np.reshape(coeff_sct3h, (R_d_AC_Sh.shape[0], R_d_AC_Sh.shape[1]))

#plt.subplot(221), plt.imshow(DC_plot), plt.title('DC'), plt.xticks([]), plt.yticks([])
#plt.colorbar()

#plt.subplot(222), plt.imshow(AC_plot), plt.title('AC'), plt.xticks([]), plt.yticks([])
#plt.colorbar()


plt.subplot(223), plt.imshow(abs_plot3h), plt.title('Absorption $\mu_a$'), plt.xticks([]),  plt.yticks([]), plt.clim(0, 0.15)
plt.colorbar()

plt.subplot(224), plt.imshow(sct_plot3h), plt.title('Reduced Scattering $\mu_s$`'), plt.xticks([]),  plt.yticks([])
plt.colorbar()


CBP:  336
num1:  336
LBP:  308
RBP:  364
Difference:  28


In [62]:
#performing ssop hilbert on image i took in lab with bench top system
#Georges way of doing hilbert
#SSOP to get optical properties
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
from unwrap import unwrap
from mpl_toolkits import mplot3d
import matplotlib
import scipy
import scipy.ndimage
from scipy.signal import hilbert
from PIL import Image
from scipy.interpolate import griddata

%matplotlib qt

im1 = (cv2.imread('Image1.jpg',0)).astype(np.double)

height = int(im1.shape[0]) 
width = int(im1.shape[1])
row = int(im1.shape[0]/2)
col = int(im1.shape[1]/2) 

f1 = np.fft.fft2(im1) #2D DFT
#for high pass filter
FT1 = np.fft.fftshift(f1)
FT1p = 20*np.log(np.abs(FT1))
#for low pass filter
FT2 = np.fft.fftshift(f1)
FT2p = 20*np.log(np.abs(FT2))


FT_pd1 = pd.DataFrame(FT1p) #whole image
cbp1 = (np.where(FT_pd1 == np.max(FT1p))[1][0])
print("CBP: ", cbp1)
FT_maxrow1 = FT1p[np.where(FT_pd1 == np.max(FT1p))[1][0]]
num1 = np.where(FT_pd1 == np.max(FT1p))[1][0]
print("num1: ", num1)
lbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[0:num1-5]))[0][0] #left bright point
print("LBP: ", lbp1)
rbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[num1+4:width]))[0][0] #right bright point
print("RBP: ", rbp1)
diff1 = int(rbp1 - cbp1)
print("Difference: ", diff1)

#FOR PLOTTING
#HPF
mask0 = FT1p 
#mask0[0:height, cbp1-diff1:cbp1+diff1] = 0
mask0[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0

#plt.subplot(121), plt.imshow(mask0)

#FOR IMAGE
#HIGH PASS FILTER
mask1 = FT1 
#mask1[0:height, cbp1-diff1:cbp1+diff1] = 0
mask1[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0


IFT = np.fft.ifft2(np.fft.ifftshift(mask1)) #inverse FFT

Real = np.real(IFT)
Imag = np.imag(IFT)


Hilb_r = scipy.signal.hilbert(Real, axis=1)
Hilb_i = scipy.signal.hilbert(Imag, axis=1)



#Signal = Real_2 + 1j*(Imag_2)#).astype(np.double))
#Signal_abs = np.abs(Signal)
Signal = (Hilb_r + 1j*Hilb_i)/2
print(Signal)

#plt.subplot(122), plt.imshow(np.abs(Signal))
#plt.imshow(np.abs(Signal)), plt.colorbar()
AC_sig = np.abs(Signal)/(np.max(np.abs(Signal))/0.3)
plt.subplot(221), plt.imshow(AC_sig), plt.title('AC'), plt.colorbar(), plt.xticks([]), plt.yticks([])

#LOW PASS FILTER
maskl2 = FT2
maskl2[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0
maskl2[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0

IFT2 = np.abs((np.fft.ifft2(np.fft.ifftshift(maskl2))))
DC_plot = IFT2/np.max(IFT2)
plt.subplot(222), plt.imshow(DC_plot), plt.title('DC'), plt.colorbar(), plt.xticks([]), plt.yticks([])


xi = []
for i in range(DC_plot.shape[0]): #does row first and loops through all columns, moves onto next row
    for j in range(DC_plot.shape[1]):
        freq = [DC_plot[i][j], AC_sig[i][j]]
        xi.append(freq)
    

#Getting array of reflectance values and corresponding optical properties
mu_a = np.arange(0, 0.3, 0.001)
mu_sp = np.arange(0.3, 3, 0.01)
n = 1.4 #refractive index of tissue 

Reflectance_AC = []

for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f2 = 0.2 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

        R_d1 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)
        
          
Reflectance_DC = []
op_mua = []
op_sp = []

for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f1 = 0 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)

        R_d2 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)
                
        op2 = mu_a[i]
        op_mua.append(op2)
        
        op3 = mu_sp[j]
        op_sp.append(op3)


points = []

for k in range(len(mu_a)*len(mu_sp)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_array = np.array(points)

op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

values1 = op_mua_array
values2 = op_sp_array


coeff_abs3 = griddata(points_array, values1, xi, method='nearest') #mua
coeff_sct3 = griddata(points_array, values2, xi, method='nearest') #musp

abs_plot3 = np.reshape(coeff_abs3, (DC_plot.shape[0], DC_plot.shape[1]))
sct_plot3 = np.reshape(coeff_sct3, (DC_plot.shape[0], DC_plot.shape[1]))


plt.subplot(223), plt.imshow(abs_plot3), plt.title('Absorption'), plt.xticks([]),  plt.yticks([]), plt.clim(0, 0.005)
plt.colorbar()

plt.subplot(224), plt.imshow(sct_plot3), plt.title('Reduced Scattering'), plt.xticks([]),  plt.yticks([])
plt.colorbar()


CBP:  240
num1:  240
LBP:  211
RBP:  268
Difference:  28
[[ -8.41826494+13.7133453j   -7.5451366  +2.67859006j
   -6.17791254 +2.66345161j ...   8.02274921 +4.74331693j
    7.35640946 +4.89901995j   7.21242571+14.34983831j]
 [ -7.53972767+13.13670215j  -6.7172896  +3.23444768j
   -5.90621081 +3.10715186j ...   8.01639915 +4.690708j
    7.31811884 +4.77767522j   7.13569968+13.85970217j]
 [ -7.2489547 +13.39207014j  -6.50355854 +3.33522473j
   -5.77528398 +3.50209614j ...   7.99602377 +4.66556835j
    7.2424429  +4.6473189j    7.49690037+13.59964635j]
 ...
 [-10.09649116+18.45362593j -10.05657094 +3.63937524j
  -10.03691112 +3.02954211j ...  10.20715727 -0.43911887j
   12.28068966 +0.84903649j  13.34772509+17.52813751j]
 [ -9.61997636+18.1027637j   -9.565129   +3.66681818j
   -9.52941921 +3.34048995j ...  10.14980012 -0.57796641j
   12.23243293 +0.70357186j  13.31087437+17.12250131j]
 [ -9.09933001+18.07195331j  -9.53890737 +4.03444592j
   -9.4958731  +3.31971015j ...  10.16610353 -0.679

In [17]:
#Fourier Transform Profilometry (FTP)
#getting phase
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
from unwrap import unwrap
import scipy
from mpl_toolkits import mplot3d
import matplotlib
from scipy.signal import hilbert

%matplotlib qt


#im1_block = (cv2.imread('Blender_sphere.png',0)).astype(np.double) #sine pattern with object 
#im2_sine = (cv2.imread('Blender_sine.png',0)).astype(np.double) #w/o object to be imaged

im1_block = (cv2.imread('PiCode_3DFinger_CROP.jpg',0)).astype(np.double) #sine pattern with object 
im2_sine = (cv2.imread('PiCode_3DSine_CROP.jpg',0)).astype(np.double) #w/o object to be imaged

height = int(im1_block.shape[0]) 
width = int(im1_block.shape[1])

f1 = np.fft.fft2(im1_block) #2D DFT
FT1 = np.fft.fftshift(f1)
FT1p = 20*np.log(np.abs(FT1))
FT_pd1 = pd.DataFrame(FT1p) #whole image
cbp1 = (np.where(FT_pd1 == np.max(FT1p))[1][0])
print("CBP: ", cbp1)
FT_maxrow1 = FT1p[np.where(FT_pd1 == np.max(FT1p))[1][0]]
num1 = np.where(FT_pd1 == np.max(FT1p))[1][0]
print("num1: ", num1)
lbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[0:num1-5]))[0][0] #left bright point
print("LBP: ", lbp1)
rbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[num1+5:width]))[0][1] #right bright point
print("RBP: ", rbp1)
diff1 = int(rbp1 - cbp1)
print("Difference: ", diff1)


FT1[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0
IFT = np.fft.ifft2(np.fft.ifftshift(FT1))

Real1 = np.real(IFT)
Imag1 = np.imag(IFT)

Hilb_r1 = scipy.signal.hilbert(Real1, axis=1)
Hilb_i1 = scipy.signal.hilbert(Imag1, axis=1)

Signal1 = (Hilb_r1 + 1j*Hilb_i1)/2 #block

#sine pattern
f2 = np.fft.fft2(im2_sine) #2D DFT
FT2 = np.fft.fftshift(f2) 
FT2p = 20*np.log(np.abs(FT2))

FT2[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0
IFT2 = np.fft.ifft2(np.fft.ifftshift(FT2))

Real2 = np.real(IFT2)
Imag2 = np.imag(IFT2)

Hilb_r2 = scipy.signal.hilbert(Real2, axis=1)
Hilb_i2 = scipy.signal.hilbert(Imag2, axis=1)

Signal2 = (Hilb_r2 + 1j*Hilb_i2)/2



fi = np.imag(np.log(Signal1*np.conj(Signal2)))
#plt.subplot(131), plt.imshow(fi), plt.colorbar(), plt.title('Phase')
unwrap = unwrap(fi, wrap_around_axis_0=False, wrap_around_axis_1=False, wrap_around_axis_2=False)
#plt.subplot(132), plt.imshow(np.abs(unwrap)), plt.colorbar(), plt.title('Abs(Unwrapped Phase)'), plt.clim(0,3)


phase_h = []
'''
#FOR BLENDER SETUP3
for j in range(im1_block.shape[0]):
    for i in range(im1_block.shape[1]):
        l_0 = 8000 #distance from camera to reference frame[mm]
        ph = np.abs(unwrap[j][i]) #calculated phase
        f_0 = 0.01 #/mm - fundamental frequency of observed grating image
        d = 560 #distance between projector and camera
        phase_height = (l_0*ph)/(ph-2*np.pi*f_0*d)
        phase_h.append(phase_height)
        
        
height = np.reshape(phase_h, (im1_block.shape[0],im1_block.shape[1]))
absheight = np.abs(height)
for i in range(im1_block.shape[0]):
    for j in range(im1_block.shape[1]):
        if absheight[i][j] > 820:
            absheight[i][j] = 0
            
'''

for j in range(im1_block.shape[0]):
    for i in range(im1_block.shape[1]):
        l_0 = 200 #distance from camera to reference frame[mm]
        ph = np.abs(unwrap[j][i]) #calculated phase
        f_0 = 0.2 #/mm - fundamental frequency of observed grating image
        d = 30 #distance between projector and camera
        phase_height = (l_0*ph)/(ph-2*np.pi*f_0*d)
        phase_h.append(phase_height)
        
height = np.reshape(phase_h, (im1_block.shape[0],im1_block.shape[1]))
absheight = np.abs(height)

for i in range(im1_block.shape[0]):
    for j in range(im1_block.shape[1]):
        if absheight[i][j] > 45:
            absheight[i][j] = 0
        
        
#plt.subplot(133), plt.imshow(absheight), plt.colorbar(), plt.xticks([]), plt.yticks([]), plt.clim(0,5)



fig = plt.figure()
ax = plt.axes(projection='3d')
x = np.linspace(0, im1_block.shape[1], im1_block.shape[1])
y = np.linspace(0, im1_block.shape[0], im1_block.shape[0])
X, Y = np.meshgrid(x, y)
ax.view_init(40, 270) #(q,r) q degrees above x-y plane and rotated r degrees counter clockwise around z #40,270
ax.plot_surface(X, Y, absheight, rstride=1, cstride=1, edgecolor='none', cmap = 'viridis'),ax.set_xlabel('x'), ax.set_ylabel('y'), ax.set_zlabel('z')


CBP:  500
num1:  500
LBP:  480
RBP:  520
Difference:  20


(<mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x21495c60a48>,
 Text(0.5, 0, 'x'),
 Text(0.5, 0, 'y'),
 Text(0.5, 0, 'z'))